<center><h1>Kelly Choy</h1></center>
<br>
<center><font size="4">Gather the Yelp Data </font></center>


## Gather the dataset

First need to download yelp dataset here: https://www.yelp.com/dataset. It takes around 6 mins to download the whole file. I am unable to attached a zip/compressed file because it is too big to upload. I tried uploading any csv (zip version) I can to the folder. We will be focusing on the business, users and reviews json files

### Import Packages

In [1]:
import json
import pandas as pd
import pandas as pd
import nltk         #Natural Language Toolkit (NLTK) library
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

#### Cleaning the Business.json file

In [8]:
#cleaning up the business.json file  - first checking the details and columns of business file
with open("yelp_business.json", "r", encoding="utf-8") as file:
    data = [json.loads(line) for line in file]

restaurants = [entry for entry in data if entry.get("categories") and "Restaurants" in entry["categories"]]

# Print the first 5 rows
for entry in restaurants[:5]:
    print(json.dumps(entry, indent=2))  # Pretty-print the JSON data

{
  "business_id": "MTSW4McQd7CbVtyjqoe9mw",
  "name": "St Honore Pastries",
  "address": "935 Race St",
  "city": "Philadelphia",
  "state": "PA",
  "postal_code": "19107",
  "latitude": 39.9555052,
  "longitude": -75.1555641,
  "stars": 4.0,
  "review_count": 80,
  "is_open": 1,
  "attributes": {
    "RestaurantsDelivery": "False",
    "OutdoorSeating": "False",
    "BusinessAcceptsCreditCards": "False",
    "BusinessParking": "{'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False}",
    "BikeParking": "True",
    "RestaurantsPriceRange2": "1",
    "RestaurantsTakeOut": "True",
    "ByAppointmentOnly": "False",
    "WiFi": "u'free'",
    "Alcohol": "u'none'",
    "Caters": "True"
  },
  "categories": "Restaurants, Food, Bubble Tea, Coffee & Tea, Bakeries",
  "hours": {
    "Monday": "7:0-20:0",
    "Tuesday": "7:0-20:0",
    "Wednesday": "7:0-20:0",
    "Thursday": "7:0-20:0",
    "Friday": "7:0-21:0",
    "Saturday": "7:0-21:0",
    "Sunday": "7:0-21:0"


In [3]:
# Create a DataFrame from the filtered data
df = pd.DataFrame(restaurants)
# Normalize the "attributes" column
df = pd.concat([df.drop(['attributes'], axis=1), pd.json_normalize(df['attributes'])], axis=1)
# df = df.drop("hours", axis=1)   #drop hours column

columns_to_drop = [
    'RestaurantsDelivery', 'OutdoorSeating', 'BusinessAcceptsCreditCards', 'BusinessParking',
    'BikeParking', 'RestaurantsTakeOut', 'ByAppointmentOnly', 'WiFi', 'Alcohol', 'Caters',
    'RestaurantsAttire', 'RestaurantsReservations', 'Ambience', 'GoodForKids', 'CoatCheck',
    'DogsAllowed', 'RestaurantsTableService', 'RestaurantsGoodForGroups', 'WheelchairAccessible',
    'HasTV', 'DriveThru', 'NoiseLevel', 'GoodForMeal', 'BusinessAcceptsBitcoin', 'Smoking',
    'Music', 'GoodForDancing', 'BestNights', 'BYOB', 'Corkage', 'BYOBCorkage',
    'RestaurantsCounterService', 'Open24Hours', 'AgesAllowed', 'DietaryRestrictions',
    'HairSpecializesIn', 'AcceptsInsurance','hours'
]

# Drop the specified columns
df = df.drop(columns=columns_to_drop)

# Display the modified DataFrame
# print(df.head())

column_names = df.columns.tolist()

# Print the column names
# print(column_names)

output_csv_file = "business_restaurants.csv"
df.to_csv(output_csv_file, index=False)

print(f"DataFrame has been saved to {output_csv_file}")

DataFrame has been saved to business_restaurants.csv


In [11]:
#getting only CA state of the restaurants
# Load the business restaurant CSV file into a DataFrame
df = pd.read_csv("business_restaurants.csv")

#get only the california restaurants
ca_restaurants = df[df['state'] == 'CA']


output_csv_file = "CA_business_restaurants.csv"
ca_restaurants.to_csv(output_csv_file, index=False)

print(f"Filtered DataFrame has been saved to {output_csv_file}")

#Notes: Results show mainly SoCal region - goleta, santa barbara, IV 

Filtered DataFrame has been saved to CA_business_restaurants.csv


#### Merging the Business Json File w/ Reviews file

In [15]:
# Load the "CA_business_restaurants.csv" into a DataFrame --> runs for about 15 minutes
df_business = pd.read_csv("CA_business_restaurants.csv")

# Load the "yelp_review.json" file into a DataFrame (assuming it's in JSON format)
df_reviews = pd.read_json("yelp_review.json", lines=True)

# Perform a left join on the "business_id" column
merged_busReview_df = pd.merge(df_business, df_reviews, on="business_id", how="left")

# Specify the path and filename for the new CSV file
output_csv_file = "merged_business_reviews.csv"

# Save the merged DataFrame to a CSV file
merged_busReview_df.to_csv(output_csv_file, index=False)

print(f"Merged Business and Reviews has been saved to {output_csv_file}")

Merged Business and Reviews has been saved to merged_business_reviews.csv


In [18]:
#finding the details that's in the user file
# Load the "yelp_user.json" file into a DataFrame (assuming it's in JSON format)
with open("yelp_user.json", "r", encoding="utf-8") as file:
    data = [json.loads(line) for line in file]

# Create a DataFrame from the loaded JSON data
df = pd.DataFrame(data)

# Get the values of the first row (the first user)
first_user_values = df.iloc[0]

# Print the values of the first user - checking the details
print(first_user_values)

user_id                                          qVc8ODYU5SZjKXVBgXdI7w
name                                                             Walker
review_count                                                        585
yelping_since                                       2007-01-25 16:47:26
useful                                                             7217
funny                                                              1259
cool                                                               5994
elite                                                              2007
friends               NSCy54eWehBJyZdG2iE84w, pe42u7DcCH2QmI81NX-8qA...
fans                                                                267
average_stars                                                      3.91
compliment_hot                                                      250
compliment_more                                                      65
compliment_profile                                              

#### Creating a final merged dataset that has business, reviews and User information into 1 csv file

In [2]:
#combining the merged dataset we had of business and review with UserID info
# Load the "yelp_user.json" file into a DataFrame (assuming it's in JSON format)
with open("yelp_user.json", "r", encoding="utf-8") as file:
    data = [json.loads(line) for line in file]

# Create a DataFrame from the loaded JSON data
userID_dataset = pd.DataFrame(data)

# List of columns to remove
columns_to_remove = [
    'compliment_hot', 'compliment_more', 'compliment_profile', 'compliment_cute',
    'compliment_list', 'compliment_note', 'compliment_plain', 'compliment_cool',
    'compliment_funny', 'compliment_writer', 'compliment_photos', 'friends'
]

# Drop the specified columns in the DataFrame without saving to a file
userID_dataset.drop(columns=columns_to_remove, inplace=True)

# Load the "merged_business_reviews.csv" into a DataFrame
merged_df = pd.read_csv("merged_business_reviews.csv")

# Perform a left join on "user_id"
result_df = pd.merge(merged_df, userID_dataset, on="user_id", how="left")

# Specify the path and filename for the new CSV file
output_csv_file = "final_dataset.csv"

# Save the merged DataFrame to the CSV file
result_df.to_csv(output_csv_file, index=False)

print(f"Merged DataFrame has been saved to {output_csv_file}")

Merged DataFrame has been saved to final_dataset.csv


#### Checking the columns of our dataset we created

In [2]:
#get the column names of the dataset
# Read the CSV file directly
dataset = pd.read_csv("final_dataset.csv", encoding="utf-8")
column_names = dataset.columns
print(column_names)

Index(['business_id', 'name_x', 'address', 'city', 'state', 'postal_code',
       'latitude', 'longitude', 'stars_x', 'review_count_x', 'is_open',
       'categories', 'RestaurantsPriceRange2', 'HappyHour', 'review_id',
       'user_id', 'stars_y', 'useful_x', 'funny_x', 'cool_x', 'text', 'date',
       'name_y', 'review_count_y', 'yelping_since', 'useful_y', 'funny_y',
       'cool_y', 'elite', 'fans', 'average_stars'],
      dtype='object')


In [3]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

dataset = pd.read_csv("final_dataset.csv")

# Additional stop words
custom_stop_words = set(['food', 'foods', 'wa'])

# Lemmatizer - researched that Lemmatizer is a linguistic tool or algorithm that reduces words to their base or root form. (useful when it comes to singular/plural forms)
lemmatizer = WordNetLemmatizer()

#text preprocessing with lemmatization
def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()
    
    # Remove punctuation and special characters
    text = ''.join([char for char in text if char not in string.punctuation])
    
    # Tokenization
    tokens = word_tokenize(text)
    
    # Lemmatization  
    lemmatized_tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    # Remove stop words (including custom stop words)
    stop_words = set(stopwords.words('english') + list(custom_stop_words))
    lemmatized_tokens = [word for word in lemmatized_tokens if word not in stop_words]
    
    return ' '.join(lemmatized_tokens)

# Applying the preprocessing function to the 'text' column
dataset['cleaned_text'] = dataset['text'].apply(preprocess_text)

# Print the cleaned text column
print(dataset['cleaned_text'])

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/kellychoy/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/kellychoy/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/kellychoy/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


0         great addition funk zone grab bite grab tastin...
1         possibly best breakfast sandwich ever biscuit ...
2         pricey ham cheese croissant 550 anywhere santa...
3         kidding without doubt best ive ever life bread...
4         super slow service salad good really priced wo...
                                ...                        
211743    one new favorite pizza joint santa barbara kno...
211744    ordered vegan chicken daiya cheese waited 15 m...
211745    worst pizza ive ever ever eatenwe ordered simp...
211746    one best pizza sbseriously regular spot friend...
211747    wanted calzone getting around time many pizza ...
Name: cleaned_text, Length: 211748, dtype: object


In [4]:
#print dataset with clean text column
dataset 

,business_id,name_x,address,city,state,postal_code,latitude,longitude,stars_x,review_count_x,...,name_y,review_count_y,yelping_since,useful_y,funny_y,cool_y,elite,fans,average_stars,cleaned_text
0,IDtLPgUrqorrpqSLdfMhZQ,Helena Avenue Bakery,"131 Anacapa St, Ste C",Santa Barbara,CA,93101,34.414445,-119.690672,4.0,389,...,Dale,17,2015-11-01 03:44:30,14,0,5,NaN,0,4.06,great addition funk zone grab bite grab tastin...
1,IDtLPgUrqorrpqSLdfMhZQ,Helena Avenue Bakery,"131 Anacapa St, Ste C",Santa Barbara,CA,93101,34.414445,-119.690672,4.0,389,...,Simone,4,2016-03-07 15:22:08,1,0,0,NaN,0,5.00,possibly best breakfast sandwich ever biscuit ...
2,IDtLPgUrqorrpqSLdfMhZQ,Helena Avenue Bakery,"131 Anacapa St, Ste C",Santa Barbara,CA,93101,34.414445,-119.690672,4.0,389,...,May,91,2010-08-01 05:16:16,37,5,22,NaN,12,3.96,pricey ham cheese croissant 550 anywhere santa...
3,IDtLPgUrqorrpqSLdfMhZQ,Helena Avenue Bakery,"131 Anacapa St, Ste C",Santa Barbara,CA,93101,34.414445,-119.690672,4.0,389,...,Koi Ling,317,2009-10-10 01:27:24,403,115,75,NaN,5,3.92,kidding without doubt best ive ever life bread...
4,IDtLPgUrqorrpqSLdfMhZQ,Helena Avenue Bakery,"131 Anacapa St, Ste C",Santa Barbara,CA,93101,34.414445,-119.690672,4.0,389,...,Jennifer,44,2011-04-26 18:18:11,55,4,5,NaN,3,3.15,super slow service salad good really priced wo...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211743,uriD7RFuHhLJeDdKaf0nFA,Pizza Guru,3534 State St,Santa Barbara,CA,93105,34.440689,-119.739681,4.0,299,...,Luke,9,2014-11-22 18:59:34,4,1,4,NaN,1,3.80,one new favorite pizza joint santa barbara kno...
211744,uriD7RFuHhLJeDdKaf0nFA,Pizza Guru,3534 State St,Santa Barbara,CA,93105,34.440689,-119.739681,4.0,299,...,Allison,87,2014-12-03 23:49:24,69,25,26,NaN,2,3.38,ordered vegan chicken daiya cheese waited 15 m...
211745,uriD7RFuHhLJeDdKaf0nFA,Pizza Guru,3534 State St,Santa Barbara,CA,93105,34.440689,-119.739681,4.0,299,...,mike,12,2011-09-13 20:40:57,3,2,1,NaN,1,3.08,worst pizza ive ever ever eatenwe ordered simp...
211746,uriD7RFuHhLJeDdKaf0nFA,Pizza Guru,3534 State St,Santa Barbara,CA,93105,34.440689,-119.739681,4.0,299,...,Tom,234,2014-01-25 18:55:41,445,101,198,"2014,2015,2016,2017,2018,2019,20,20,2021",30,4.40,one best pizza sbseriously regular spot friend...


In [5]:
# List of columns to drop
#stars_y column =  user's review rating
drop_cols = ['is_open', 'HappyHour', 'review_id', 'useful_x', 'funny_x', 'cool_x', 'date',
       'name_y', 'review_count_y', 'yelping_since', 'useful_y', 'funny_y',
       'cool_y', 'elite', 'fans', 'average_stars','categories','state','postal_code','latitude','longitude','address','review_count_x',
       'RestaurantsPriceRange2']

# Create a new DataFrame with the specified columns dropped
updated_dataset = dataset.drop(columns=drop_cols)

# Print the updated DataFrame
print("After dropping columns:")
updated_dataset

After dropping columns:


,business_id,name_x,city,stars_x,user_id,stars_y,text,cleaned_text
0,IDtLPgUrqorrpqSLdfMhZQ,Helena Avenue Bakery,Santa Barbara,4.0,bFPdtzu11Oi0f92EAcjqmg,5,What a great addition to the Funk Zone! Grab ...,great addition funk zone grab bite grab tastin...
1,IDtLPgUrqorrpqSLdfMhZQ,Helena Avenue Bakery,Santa Barbara,4.0,Ttb21DpWqphQMNSS5TXgMA,5,Possibly the best breakfast sandwich EVER. On...,possibly best breakfast sandwich ever biscuit ...
2,IDtLPgUrqorrpqSLdfMhZQ,Helena Avenue Bakery,Santa Barbara,4.0,ha2Lv7WnxvdYnRsqwL1apQ,4,Pricey ( a ham and cheese croissant was $5.50)...,pricey ham cheese croissant 550 anywhere santa...
3,IDtLPgUrqorrpqSLdfMhZQ,Helena Avenue Bakery,Santa Barbara,4.0,OVLf6NVTi7noMP1qCKr76w,5,Are you kidding me? This is without a doubt so...,kidding without doubt best ive ever life bread...
4,IDtLPgUrqorrpqSLdfMhZQ,Helena Avenue Bakery,Santa Barbara,4.0,2ykh0_q0uMLFtypLDVSqlg,2,"Super slow service, ""salads"" are not very good...",super slow service salad good really priced wo...
...,...,...,...,...,...,...,...,...
211743,uriD7RFuHhLJeDdKaf0nFA,Pizza Guru,Santa Barbara,4.0,o72LSEPHKu1oQcpXeIVlag,5,This is one of my new favorite pizza joints he...,one new favorite pizza joint santa barbara kno...
211744,uriD7RFuHhLJeDdKaf0nFA,Pizza Guru,Santa Barbara,4.0,u30sv3s4V2tz5VOgNQYtgg,4,Ordered vegan chicken and Daiya cheese and onl...,ordered vegan chicken daiya cheese waited 15 m...
211745,uriD7RFuHhLJeDdKaf0nFA,Pizza Guru,Santa Barbara,4.0,Mp97BPH2Y7O1DsFt6wFp4Q,1,This was the worst pizza I've ever had ever ea...,worst pizza ive ever ever eatenwe ordered simp...
211746,uriD7RFuHhLJeDdKaf0nFA,Pizza Guru,Santa Barbara,4.0,06egjh3nrEb7EXX1VYZOCg,5,One of the best pizzas in SB....seriously!!!\n...,one best pizza sbseriously regular spot friend...


In [6]:
#print out a new csv for text vectorization into a new csv file
output_csv_file = "text_vectorization.csv"

# Save the filtered DataFrame to the CSV file
updated_dataset.to_csv(output_csv_file, index=False)

print(f"DataFrame has been saved to {output_csv_file}")

DataFrame has been saved to text_vectorization.csv


### Next go to the comm599_Topic_Modeling_step.ipynb file for more code/next steps to the project